思路：

generate_position_labels：
输入：一个paragraph string
输出：一个嵌套的list [[x, y, w, h, string], [x, y, w, h, string], ...]

make_images

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import requests
import re
import cv2
import os
from PIL import Image
import random
import torch

In [25]:
def word_list(file):
    text = open(file, 'r').read()
    
    # Define the acceptable characters
    uppercase = [chr(i) for i in range(65, 91)]
    lowercase = [chr(i) for i in range(97, 123)]
    digits = [i for i in range(10)]
    word_list = []
    temp_word = ""

    for char in text:
        if char in uppercase or char in lowercase or char in digits:
            temp_word += char
        elif char == " " or char == "\n":
            word_list.append(temp_word)
            temp_word = ""
        elif char == ".":
            word_list.append(temp_word)
            word_list.append(".")
            temp_word = ""
        else:
            if temp_word:
                word_list.append(temp_word)
                temp_word = ""
    
    word_list = [word for word in word_list if word != ""]
    return word_list


In [26]:
word_list = word_list("gatsby.txt")
print(word_list[:100])

['The', 'Project', 'Gutenberg', 'eBook', 'of', 'The', 'Great', 'Gatsby', 'This', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'United', 'States', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.', 'You', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'Project', 'Gutenberg', 'License', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www', '.', 'gutenberg', '.', 'org', '.', 'If', 'you', 'are', 'not', 'located', 'in', 'the', 'United', 'States', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'eBook', '.', 'Title', 'The']


In [2]:
def generate_labels(paragraph):
    # Define the scale
    line_height = 28

    # Define the basic variable
    labels = []
    current_x = random.randint(20, 30)
    current_y = random.randint(20, 30)
    temp_word = ""
    temp_start_x = 0

    # Loop through the paragraph
    for char in paragraph:
        space_width = random.randint(20, 30)    # Random space width
        # If meet with Space or Enter
        if char == " " or char == "\n":
            if temp_word:
                labels.append([temp_start_x, current_y, current_x - temp_start_x, line_height, temp_word])  # [top-left x, top-left y, width, height, word]
                temp_word = ""
            if char == " ":
                current_x += space_width
            if char == "\n":
                current_y += line_height + random.randint(20, 35)
                current_x = random.randint(20, 30)
        # If meet with Dot
        elif char == ".":
            if temp_word:
                labels.append([temp_start_x, current_y, current_x - temp_start_x, line_height, temp_word])
                temp_word = ""
            labels.append([current_x, current_y, 10, line_height, "."])
            current_x += 10 + space_width
        # Appending the characters into the list
        else:
            if not temp_word:
                temp_start_x = current_x
            temp_word += char
            current_x += 28
    # Append the last word
    if temp_word:
        labels.append([temp_start_x, current_y, current_x - temp_start_x, line_height, temp_word])


    return labels

In [11]:
def process_paragraph(file):
    # Read the txt file
    texts = open(file, "r").read()

    # Define the basic variables
    word_count = 0
    paragraphs = []
    current_paragraph = ""
    current_word = ""

    # Define the acceptable characters
    uppercase = [chr(i) for i in range(65, 91)]
    lowercase = [chr(i) for i in range(97, 123)]
    digits = [i for i in range(10)]

    # Loop through the text
    for char in texts:
        if char.isalnum():
            current_word += char
        else:
            if current_word:
                word_count += 1
                current_paragraph += current_word
                current_word = ""
            if char == "\n":
                current_paragraph += "\n"
            elif char == "\r":
                continue
            elif re.match(r'[^\w\s]', char) or char == ' ':
                current_paragraph += char

        # If the word count reaches 90, append the paragraph
        if word_count == 90:
            paragraphs.append(current_paragraph)
            current_paragraph = ""
            word_count = 0

    # Append the last paragraph
    if current_paragraph or current_word:
        current_paragraph += current_word
        paragraphs.append(current_paragraph)
    
    # Clean the paragraphs
    cleaned_paragraphs = []

    # Loop through the paragraphs
    for paragraph in paragraphs:
        temp_paragraph = ""
        for char in paragraph:
            if char in uppercase or char in lowercase or char in digits or char == "." or char == "\n" or char == " ":
                temp_paragraph += char
            else:
                temp_paragraph += " "
        cleaned_paragraphs.append(temp_paragraph)
    
    return cleaned_paragraphs

In [15]:
paragraph = process_paragraph("test_paragraph.txt")

print(paragraph)

['The practical thing was to find rooms in the city  but it was a warm\nseason  and I had just left a country of wide lawns and friendly\ntrees  so when a young man at the office suggested that we take a\nhouse together in a commuting town  it sounded like a great idea. He\nfound the house  a weather beaten cardboard bungalow at eighty a\nmonth  but at the last minute the firm ordered him to Washington  and\nI went out to the country alone. I had a dog ', 'at least I had him for a\nfew days until he ran away and an old Dodge and a Finnish woman  who\nmade my bed and cooked breakfast and muttered Finnish wisdom to\nherself over the electric stove.']


In [18]:
def resize_word(image):
    # Get the image size
    width, height = image.shape[1], image.shape[0]

    # Define the scale
    new_length = max(width, height)

    # Define the new image
    new_image = np.ones((new_length, new_length), dtype=np.uint8) * 255

    # Place the image into the new image
    new_image[(new_length - height) // 2:(new_length - height) // 2 + height, (new_length - width) // 2:(new_length - width) // 2 + width] = image

    # Resize the image
    new_image = cv2.resize(new_image, (128, 128))
    
    return new_image

In [17]:
# Read from test_paragraph.txt
labels = generate_labels(paragraph[0])

# Print all the labels
for label in labels:
    print(label)

[22, 20, 84, 28, 'The']
[134, 20, 252, 28, 'practical']
[408, 20, 140, 28, 'thing']
[568, 20, 84, 28, 'was']
[678, 20, 56, 28, 'to']
[759, 20, 112, 28, 'find']
[896, 20, 140, 28, 'rooms']
[1057, 20, 56, 28, 'in']
[1137, 20, 84, 28, 'the']
[1244, 20, 112, 28, 'city']
[1403, 20, 84, 28, 'but']
[1515, 20, 56, 28, 'it']
[1593, 20, 84, 28, 'was']
[1699, 20, 28, 28, 'a']
[1756, 20, 112, 28, 'warm']
[22, 72, 168, 28, 'season']
[244, 72, 84, 28, 'and']
[350, 72, 28, 28, 'I']
[401, 72, 84, 28, 'had']
[506, 72, 112, 28, 'just']
[644, 72, 112, 28, 'left']
[778, 72, 28, 28, 'a']
[832, 72, 196, 28, 'country']
[1051, 72, 56, 28, 'of']
[1128, 72, 112, 28, 'wide']
[1265, 72, 140, 28, 'lawns']
[1425, 72, 84, 28, 'and']
[1532, 72, 224, 28, 'friendly']
[25, 128, 140, 28, 'trees']
[217, 128, 56, 28, 'so']
[301, 128, 112, 28, 'when']
[437, 128, 28, 28, 'a']
[490, 128, 140, 28, 'young']
[659, 128, 84, 28, 'man']
[765, 128, 56, 28, 'at']
[842, 128, 84, 28, 'the']
[949, 128, 168, 28, 'office']
[1146, 128, 252

In [19]:
def transfer_tensor(word):
    # Define the basic value
    STX = 2
    ETX = 3
    word_info = torch.zeros(64,128,dtype=torch.long)

    # Define the start and end
    word_info[0, STX] = 1
    for i, c in enumerate(word, 1):
        word_info[i, ord(c)] = 1
    word_info[i + 1, ETX] = 1

    return word_info